Imports

In [1]:
import sys
sys.path.insert(0, "/home/m4pc/m4v2-code/m4_ws/src/morphing_lander/")
from pathlib import Path
import torch
import matplotlib.pyplot as plt 
from morphing_lander.data_utils.data_utils import get_data_from_rosbag, train_val_split
from morphing_lander.cvae.train import TrainConfig, train

Get dataset ready for training

In [2]:
# get data from rosbag
bag_path    = '/home/m4pc/m4v2-code/m4_ws/rosbag2_2024_05_21-23_05_12'
type_path   = '/home/m4pc/m4v2-code/m4_ws/src/custom_msgs/msg/MPCStatus.msg'
topic_name  = 'mpc_status'

percent_val = 0.1
data = get_data_from_rosbag(bag_path,type_path,topic_name)

# split data into train and validation set
train_data_path = "/home/m4pc/m4v2-code/m4_ws/src/morphing_lander/data/train_data.pt"
val_data_path = "/home/m4pc/m4v2-code/m4_ws/src/morphing_lander/data/val_data.pt"

state_idx = [2,3,4,5,6,7,8] # train on z,thetaz,thetay,thetax,xdot,ydot,zdot # also phi but this is included automatically
input_idx = [0,1,2,3]       # train on 4 thruster PWMs
include_phi = False
train_data, val_data = train_val_split(data,percent_val,state_idx,input_idx,include_phi)

# save train and validation data for future use
torch.save(train_data, train_data_path)
torch.save(val_data, val_data_path)

print(f"training data points: {train_data.shape[0]}")
print(f"validation data points: {val_data.shape[0]}")


training data points: 12634
validation data points: 1403


In [3]:
import plotly.graph_objects as go

d     = data['d']
f_res = data['f_res']
x_vec = data['x_vec']
x_ref_vec = data['x_ref_vec']
x_next_vec = data['x_next_vec']
u_vec = data['u_vec']
phi_vec = data['phi_vec']
t_vec = data['t_vec']
dt_vec = data['dt_vec']

plt.figure()
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=t_vec[1:],
        y=x_vec[:,2],
        name="Actual",
        line=dict(color="blue"),
        # fill="blue",
    )
)

fig.add_trace(
    go.Scatter(
        x=t_vec[1:],
        y=x_next_vec[:,2],
        name="Model one step prediction",
        line=dict(color="red"),
    )
)

<Figure size 640x480 with 0 Axes>

Specify training configuration

In [4]:
# Specify the path for the learned models
output_path = Path(
    "/home/m4pc/m4v2-code/m4_ws/src/morphing_lander/learned_models/"
)

train_config = TrainConfig(
    output_dim=12,
    latent_dim=6,
    cond_dim=11,
    encoder_layers=[32, 32],
    decoder_layers=[32, 32],
    prior_layers=[32, 32],
    batches_per_epoch=10,
    epochs=300,
    step_size=50,  # steps per decay for lr scheduler
    gamma=0.75,    # multiplicative decay for lr scheduler
    lr=5e-4,
    save_epochs=10,
    val_epochs=10,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

print(f"Device is: {train_config.device}")

Device is: cuda


Train

In [5]:

model = train(train_data_path,val_data_path,train_config,output_path)

Epoch 0: train_loss=12.985023498535156, val_loss=19.28577995300293
Epoch 10: train_loss=15.037355422973633, val_loss=15.36937665939331
Epoch 20: train_loss=10.731014251708984, val_loss=12.087880611419678
Epoch 30: train_loss=2.577676296234131, val_loss=4.118114233016968
Epoch 40: train_loss=0.5889266729354858, val_loss=1.7675621509552002
Epoch 50: train_loss=5.6127705574035645, val_loss=-0.3861895501613617
Epoch 60: train_loss=-2.0581119060516357, val_loss=-0.566809294745326
Epoch 70: train_loss=13.299871444702148, val_loss=-2.3516111373901367
Epoch 80: train_loss=-4.459570407867432, val_loss=-2.6252416372299194
Epoch 90: train_loss=0.6009624004364014, val_loss=-1.5534443855285645
Epoch 100: train_loss=1.9766857624053955, val_loss=0.35347071290016174
Epoch 110: train_loss=0.8959701061248779, val_loss=-2.130066692829132
Epoch 120: train_loss=1.9922490119934082, val_loss=-2.941509246826172
Epoch 130: train_loss=-2.380013942718506, val_loss=-1.2672793865203857
Epoch 140: train_loss=-0.119